Кобзарь О.С. Хабибуллин Р.А. 2019 г.

## Данное решение находится в стадии разработки. Нет  учета изменения свойств флюида в затрубном пространстве. Нет уверенности в правильности формул.

### КРТ. Температурная корреляция Хасан Кабир
#### Последовательный расчет градиента давления (Беггз Брил) и градиента температуры

Импорт необходимых модулей

In [1]:
import sys
sys.path.append('../')


import uniflocpy.uWell.uPipe as Pipe
import uniflocpy.uTools.data_workflow as tool
import uniflocpy.uTools.uconst as uc

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools

init_notebook_mode(connected=True)

создание экземляра класса Pipe и хранилищ данных

In [2]:
tube = Pipe.Pipe()
tube_data = tool.Data()
fluid_flow_data = tool.Data()
temp_cor_data = tool.Data()
fluid_data = tool.Data()

Задание необходимых параметров скважины (конструкция трубы, начальные условия, свойства потока, время работы)

In [3]:
p_init_bar = 90
t_fluid_init_c = 91
t_earth_init_c = 91
q_liq_m3day = 50
watercut_perc = 20
d_tube_m = 0.120
work_time_sec = 2 * 60 * 60
geo_grad_cm = 0.027
depth_m = 3000
step_m = 10

Передача параметров скважины в класс Pipe

In [4]:
tube.fluid_flow.qliq_on_surface_m3day = q_liq_m3day
tube.fluid_flow.fw_on_surface_perc = watercut_perc
tube.fluid_flow.d_m = d_tube_m

tube.section_casing = True

tube.t_earth_init_c = t_earth_init_c
tube.time_sec = work_time_sec

Проверка для расчета градиента давления и температуры при начальных условиях

In [5]:
tube.calc_p_grad_pam(p_init_bar, t_fluid_init_c)

9568.240555220334

In [6]:
tube.calc_t_grad_cm(p_init_bar, t_fluid_init_c)

0.007272985550726305

Создание хранилищ для основных параметров работы

In [7]:
p_list_bar = [p_init_bar]
t_list_c = [t_fluid_init_c]
h_list_m = [depth_m]
t_earth_list_c = [t_earth_init_c]
distance_from_bottomhole = [0]

Основной расчет через цикл

In [8]:
for i in range(int(depth_m / step_m)):
    if p_list_bar[-1] > 2:
        p_grad_pam = tube.calc_p_grad_pam(p_list_bar[-1], t_list_c[-1])
        t_grad_cm = tube.calc_t_grad_cm(p_list_bar[-1], t_list_c[-1])
        

        
        
        p_bar = p_list_bar[-1] - uc.Pa2bar(p_grad_pam) * step_m
        t_c = t_list_c[-1] - t_grad_cm * step_m
        t_earth_c = t_earth_list_c[-1] - geo_grad_cm * step_m
        h_m = h_list_m[-1] - step_m
        distance_m = distance_from_bottomhole[-1] + step_m
        
        
        tube_data.get_data(tube)
        fluid_flow_data.get_data(tube.fluid_flow)
        temp_cor_data.get_data(tube.temp_cor)
        fluid_data.get_data(tube.fluid_flow.fl)
        
        fluid_data.h_list.append(distance_m)
        
        h_list_m.append(h_m)
        p_list_bar.append(p_bar)
        t_list_c.append(t_c)
        t_earth_list_c.append(t_earth_c)
        distance_from_bottomhole.append(distance_m)
        
        tube.t_earth_init_c = t_earth_c
        

Функции, для удобства построения графиков

In [9]:
def plot():
    layout = dict(title =  'L(P бар, T C)',  yaxis=dict(title='Расстояние от забоя, м'))
    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')

def trace(data, number_param):
    tracep = go.Scattergl(
        x = data.get_values(number_param),
        y = fluid_data.h_list,
        name = data.get_name(number_param),
        mode = 'markers'
    )
    return tracep

In [10]:
trace1 = go.Scattergl(
    x = t_earth_list_c,
    y = distance_from_bottomhole, 
    name = 'Геотерма, C',
    mode = 'markers'
)

trace2 = go.Scattergl(
    x = p_list_bar,
    y = distance_from_bottomhole, 
    name = 'КРД Беггз Брилл, бар',
    mode = 'markers'
)

trace3 = go.Scattergl(
    x = t_list_c,
    y = distance_from_bottomhole, 
    name = 'КРТ Хасан Кабир, С',
    mode = 'markers'
)


data = [ trace1, trace2, trace3]

plot()

In [11]:
tube_data
fluid_flow_data
temp_cor_data
fluid_data
fluid_data

In [12]:
temp_cor_data.print_all_names()

Номер 0 для получения параметра rho_earth_kgm3
Номер 1 для получения параметра cp_earth_jkgc
Номер 2 для получения параметра thermal_conduct_earth_wmc
Номер 3 для получения параметра thermal_conduct_tube_wmk
Номер 4 для получения параметра thermal_conduct_cas_wmk
Номер 5 для получения параметра thermal_conduct_cem_wmc
Номер 6 для получения параметра delta_t_an_init_c
Номер 7 для получения параметра section_casing
Номер 8 для получения параметра time_sec
Номер 9 для получения параметра p_bar
Номер 10 для получения параметра t_c
Номер 11 для получения параметра t_earth_init_c
Номер 12 для получения параметра angle_rad
Номер 13 для получения параметра d_m
Номер 14 для получения параметра r_tube_in_m
Номер 15 для получения параметра r_tube_out_m
Номер 16 для получения параметра r_cas_in_m
Номер 17 для получения параметра r_cas_out_m
Номер 18 для получения параметра r_well_m
Номер 19 для получения параметра liquid_content
Номер 20 для получения параметра mass_flowraten_kgsec
Номер 21 для по

In [13]:
trace1 = trace(temp_cor_data, 54)
trace2 = trace(temp_cor_data, 55)
trace3 = trace(temp_cor_data, 56)
trace4 = trace(temp_cor_data, 57)
trace5 = trace(temp_cor_data, 58)





data = [trace1, trace2,trace3,  trace4, trace5]

plot()

In [14]:
trace1 = trace(temp_cor_data, 45)
trace2 = trace(temp_cor_data, 46)
trace3 = trace(temp_cor_data, 47)
trace4 = trace(temp_cor_data, 48)
trace5 = trace(temp_cor_data, 49)
trace6 = trace(temp_cor_data, 50)





data = [trace1, trace2,trace3,  trace4, trace5, trace6]

plot()